## Spread Workshop


**Spread Prototypes** ease definition and management of (mainly) Option-Spreads



In [1]:
require_relative '../lib/iruby_init.rb'


----> G    points to the Gateway-Instance
----> C    points to the Connection-Instance
----> Multible accounts detected
----> G.clients  points to the User-Accounts
Namespace is IB !


true

### Plain Vanilla Spreads

`IB::Bag` is a native TWS-Contract.

`IB::Spread` extents IB::Bags. They are tradeable and are then reduced to  `IB::Bag`-Objects, whoch are transmitted  to retrieve contract-informations or to initiate the order-process.

In [2]:
spread =  IB::Spread.new symbol: 'ESTX50', currency: 'EUR', exchange: 'DTB'
# which is identical to
spread = IB::Spread.new  IB::Symbols::Index.stoxx.attributes.slice( :exchange, :symbol, :currency )

#<IB::Spread:0x0000562fdb567640 @attributes={:exchange=>"DTB", :symbol=>"ESTX50", :currency=>"EUR", :created_at=>2021-07-08 18:39:01.745357934 +0000, :right=>"", :include_expired=>false, :sec_type=>"BAG"}, @legs=[]>

The `legs`-Array is empty.
Legs are added through `IB::Spread.add_leg`

In [3]:
leg_1 = IB::Option.new( IB::Symbols::Index.stoxx.attributes.slice( :exchange, :symbol, :currency))
                 .merge( expiry: IB::Symbols::Futures.next_expiry, 
                         strike: 4000, right: :put,
                         trading_class: "OESX" )
leg_2 = leg_1.merge( right: :call )

#
#  Important: predefined legs need a valid `con_id`
#
spread.add_leg leg_1.verify.first, action: :buy
spread.add_leg leg_2.verify.first, action: :buy

# -------------------------------------------------
puts "This appears to be a straddle"
puts spread.as_table
puts spread.legs.as_table
puts spread.combo_legs.to_human
puts "Actual Market Price: " + spread.market_price.to_s

This appears to be a straddle
┌────────┬────────┬────────────┬──────────┬────────┬────────────┬───────────────┬───────┬────────┬──────────┐
│        │ symbol │ con_id     │ exchange │ expiry │ multiplier │ trading-class │ right │ strike │ currency │
╞════════╪════════╪════════════╪══════════╪════════╪════════════╪═══════════════╪═══════╪════════╪══════════╡
│ Spread │ ESTX50 │ -883641671 │   DTB    │        │     10     │               │       │        │   EUR    │
└────────┴────────┴────────────┴──────────┴────────┴────────────┴───────────────┴───────┴────────┴──────────┘
┌────────┬────────┬───────────┬──────────┬──────────┬────────────┬───────────────┬───────┬────────┬──────────┐
│        │ symbol │ con_id    │ exchange │ expiry   │ multiplier │ trading-class │ right │ strike │ currency │
╞════════╪════════╪═══════════╪══════════╪══════════╪════════════╪═══════════════╪═══════╪════════╪══════════╡
│ Option │ ESTX50 │ 441820851 │   DTB    │ 20210917 │     10     │     OESX      │  put

## Using a Spread-Prototype

All Prototypes are either fabricated or build.

* If one leg is already present as `IB::Contract`, this acts as Master to `fabricate` the spread.
* If only the underlying is available, `build from: (underlying)` is used. 


### Straddle

The procedure described above is implemented in the `IB::Straddle` prototype.


In [4]:
spread =  IB::Straddle.build from: Symbols::Index.stoxx, 
                             strike: 4000, 
                             expiry: IB::Symbols::Futures.next_expiry,
                             trading_class: "OESX"

puts spread.to_human
puts spread.as_table
puts spread.legs.as_table
puts spread.combo_legs.to_human
puts "Actual Market Price: " + spread.market_price.to_s

<Straddle ESTX50(4000.0)[Sep 2021]>
┌────────┬────────┬────────────┬──────────┬────────┬────────────┬───────────────┬───────┬────────┬──────────┐
│        │ symbol │ con_id     │ exchange │ expiry │ multiplier │ trading-class │ right │ strike │ currency │
╞════════╪════════╪════════════╪══════════╪════════╪════════════╪═══════════════╪═══════╪════════╪══════════╡
│ Spread │ ESTX50 │ -883641671 │   DTB    │        │     10     │               │       │        │   EUR    │
└────────┴────────┴────────────┴──────────┴────────┴────────────┴───────────────┴───────┴────────┴──────────┘
┌────────┬────────┬───────────┬──────────┬──────────┬────────────┬───────────────┬───────┬────────┬──────────┐
│        │ symbol │ con_id    │ exchange │ expiry   │ multiplier │ trading-class │ right │ strike │ currency │
╞════════╪════════╪═══════════╪══════════╪══════════╪════════════╪═══════════════╪═══════╪════════╪══════════╡
│ Option │ ESTX50 │ 441820851 │   DTB    │ 20210917 │     10     │     OESX      

### Strangle

To define a strangle, the strikes of the two Options are needed

In [5]:
spread =  IB::Strangle.build from: Symbols::Index.stoxx, 
                             p: 3800, c: 4200,
                             expiry: IB::Symbols::Futures.next_expiry,
                             trading_class: "OESX"

puts spread.to_human
puts spread.as_table
puts spread.legs.as_table
puts spread.combo_legs.to_human
puts "Actual Market Price: " + spread.market_price.to_s

<Strangle ESTX50(3800.0,4200.0)[Sep 2021]>
┌────────┬────────┬────────────┬──────────┬────────┬────────────┬───────────────┬───────┬────────┬──────────┐
│        │ symbol │ con_id     │ exchange │ expiry │ multiplier │ trading-class │ right │ strike │ currency │
╞════════╪════════╪════════════╪══════════╪════════╪════════════╪═══════════════╪═══════╪════════╪══════════╡
│ Spread │ ESTX50 │ -884245163 │   DTB    │        │     10     │               │       │        │   EUR    │
└────────┴────────┴────────────┴──────────┴────────┴────────────┴───────────────┴───────┴────────┴──────────┘
┌────────┬────────┬───────────┬──────────┬──────────┬────────────┬───────────────┬───────┬────────┬──────────┐
│        │ symbol │ con_id    │ exchange │ expiry   │ multiplier │ trading-class │ right │ strike │ currency │
╞════════╪════════╪═══════════╪══════════╪══════════╪════════════╪═══════════════╪═══════╪════════╪══════════╡
│ Option │ ESTX50 │ 441514433 │   DTB    │ 20210917 │     10     │     OES

### Calendar

To define a calendar-spread,  a stike and two different expiries (:front, :back) have to be specified.


In [6]:
spread =  IB::Calendar.build from: Symbols::Index.stoxx, 
                             strike: 3800, 
                             front: IB::Symbols::Futures.next_expiry,
                             back: '+3m',
                             right: :call
                            # trading_class: "OESX"


puts spread.to_human
puts spread.as_table
puts spread.legs.as_table
puts spread.combo_legs.to_human
puts "Actual Market Price: " + spread.market_price.to_s

<Calendar ESTX50 call(3800.0)[1 :Sep 2021 |+|-1 :Dec 2021  >
┌────────┬────────┬────────────┬──────────┬────────┬────────────┬───────────────┬───────┬────────┬──────────┐
│        │ symbol │ con_id     │ exchange │ expiry │ multiplier │ trading-class │ right │ strike │ currency │
╞════════╪════════╪════════════╪══════════╪════════╪════════════╪═══════════════╪═══════╪════════╪══════════╡
│ Spread │ ESTX50 │ -676443531 │   DTB    │        │     10     │               │       │        │   EUR    │
└────────┴────────┴────────────┴──────────┴────────┴────────────┴───────────────┴───────┴────────┴──────────┘
┌────────┬────────┬───────────┬──────────┬──────────┬────────────┬───────────────┬───────┬────────┬──────────┐
│        │ symbol │ con_id    │ exchange │ expiry   │ multiplier │ trading-class │ right │ strike │ currency │
╞════════╪════════╪═══════════╪══════════╪══════════╪════════════╪═══════════════╪═══════╪════════╪══════════╡
│ Option │ ESTX50 │ 441513817 │   DTB    │ 20210917 │   

### Vertical

To define a vertical-spread,  the expiry and two strikes (:buy, :sell) have to be specified.


In [7]:
spread =  IB::Vertical.build from: Symbols::Index.stoxx, 
                             expiry: IB::Symbols::Futures.next_expiry,
                             buy: 3800, sell: 3600 ,
                             right: :call,
                             trading_class: "OESX"

puts spread.to_human
puts spread.as_table
puts spread.legs.as_table
puts spread.combo_legs.to_human
puts "Actual Market Price: " + spread.market_price.to_s

<Vertical ESTX50 call(-1 :3600.0 |+|1 :3800.0  )[Sep 2021]>
┌────────┬────────┬────────────┬──────────┬────────┬────────────┬───────────────┬───────┬────────┬──────────┐
│        │ symbol │ con_id     │ exchange │ expiry │ multiplier │ trading-class │ right │ strike │ currency │
╞════════╪════════╪════════════╪══════════╪════════╪════════════╪═══════════════╪═══════╪════════╪══════════╡
│ Spread │ ESTX50 │ -883027617 │   DTB    │        │     10     │               │       │        │   EUR    │
└────────┴────────┴────────────┴──────────┴────────┴────────────┴───────────────┴───────┴────────┴──────────┘
┌────────┬────────┬───────────┬──────────┬──────────┬────────────┬───────────────┬───────┬────────┬──────────┐
│        │ symbol │ con_id    │ exchange │ expiry   │ multiplier │ trading-class │ right │ strike │ currency │
╞════════╪════════╪═══════════╪══════════╪══════════╪════════════╪═══════════════╪═══════╪════════╪══════════╡
│ Option │ ESTX50 │ 441513800 │   DTB    │ 20210917 │    

### Butterfly

To define a Butterfly, apart from expiry and right, the main strike (:strike) which is sold and the two bought option-strikes (:front, :back) have to be specified.


In [8]:
spread =  IB::Butterfly.build from: Symbols::Index.stoxx, 
                             expiry: IB::Symbols::Futures.next_expiry,
                             front: 3800, back: 4200 , strike: 4000,
                             right: :call,
                             trading_class: "OESX"

puts spread.to_human
puts spread.as_table
puts spread.legs.as_table
puts spread.combo_legs.to_human
puts "Actual Market Price: " + spread.market_price.to_s

<Butterfly ESTX50 call(1 :3800.0 |+|-2 :4000.0 |+|1 :4200.0  )[Sep 2021]>
┌────────┬────────┬─────────────┬──────────┬────────┬────────────┬───────────────┬───────┬────────┬──────────┐
│        │ symbol │ con_id      │ exchange │ expiry │ multiplier │ trading-class │ right │ strike │ currency │
╞════════╪════════╪═════════════╪══════════╪════════╪════════════╪═══════════════╪═══════╪════════╪══════════╡
│ Spread │ ESTX50 │ -1326065367 │   DTB    │        │     10     │               │       │        │   EUR    │
└────────┴────────┴─────────────┴──────────┴────────┴────────────┴───────────────┴───────┴────────┴──────────┘
┌────────┬────────┬───────────┬──────────┬──────────┬────────────┬───────────────┬───────┬────────┬──────────┐
│        │ symbol │ con_id    │ exchange │ expiry   │ multiplier │ trading-class │ right │ strike │ currency │
╞════════╪════════╪═══════════╪══════════╪══════════╪════════════╪═══════════════╪═══════╪════════╪══════════╡
│ Option │ ESTX50 │ 441513817 │   DTB 